In [6]:
import math

In [34]:
class Grid(object):
    def __init__(self):
        self.elements = range(1,19)
        self.nodes = range(1,17)

    def nodes_associated_with_element(self, element):
        square = math.floor((float(element)+1.0)/2.0)
        row = math.floor((square-.5)/3.0)
        is_upper_triangle = element%2
        lower_left_node = int(square + row)
        upper_right_node = lower_left_node + 5
        third_node = lower_left_node + 1
        if is_upper_triangle:
            third_node = upper_right_node - 1
        return sorted([lower_left_node,third_node,upper_right_node])
        


In [35]:
grid = Grid()

In [40]:
# Nodes associated with element test
assert grid.nodes_associated_with_element(1) == [1,5,6]
assert grid.nodes_associated_with_element(2) == [1,2,6]
assert grid.nodes_associated_with_element(7) == [5,9,10]
assert grid.nodes_associated_with_element(8) == [5,6,10]